### Tutorial ontf.nn.embedding_lookup_sparse

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# set params
n = 6
f = 3
k = 2

In [3]:
# build embeddings
embeddings = tf.get_variable("embeddings",shape=[n,f,k],dtype=tf.float32,
                   initializer=tf.truncated_normal_initializer(stddev=1.0,mean=0))

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
W = sess.run(embeddings)
print("Defined embeddings: \n",W)

Defined embeddings: 
 [[[-0.22190836  0.9851174 ]
  [ 0.7041001   1.1924982 ]
  [-1.0887938  -0.05284955]]

 [[ 0.6250706  -0.06330938]
  [-0.26733723  0.5267984 ]
  [-0.22102411 -0.03416568]]

 [[ 1.9543301   0.8465737 ]
  [-0.29303005 -0.9815563 ]
  [ 0.8602309  -0.0750349 ]]

 [[ 0.6513389  -0.6178217 ]
  [ 1.6611365   1.3777244 ]
  [-0.39056495 -0.21109547]]

 [[-1.4958913  -0.13084617]
  [-0.7257098   0.10666375]
  [ 1.2406721  -0.31811637]]

 [[-0.3088654   0.39020845]
  [ 1.1088725  -1.9077142 ]
  [-0.06244733 -0.2436103 ]]]


In [5]:
# define samples feature and values
feature = [0,2,5] # field_0: feat_0, field_1: feat_2, field_3: feat_3
value = [1,1,1] # value of each selected feature

# build sparse tensor
sp_feat = tf.SparseTensor(indices=[[0,0],[0,1],[0,2]],dense_shape=[1,3],values=[0,2,5])
print("sparse feature:",sess.run(tf.sparse_tensor_to_dense(sp_feat)))

sp_value = tf.SparseTensor(indices=[[0,0],[0,1],[0,2]],dense_shape=[1,3],values=[1,1,1])
print("sparse values:",sess.run(tf.sparse_tensor_to_dense(sp_value)))

sparse feature: [[0 2 5]]
sparse values: [[1 1 1]]


In [10]:
# real quad term
quad_term = 0
for i in range(f):
    for j in range(i+1,f):
        print(i,j)
        vi_fj = W[feature[i],j]
        vj_fi = W[feature[j],i]
        term = vi_fj.dot(vj_fi) * value[i] * value[j]
        print("term:",term)
        quad_term += term
print("___"*10)
print("result:",quad_term)

0 1
term: 2.3855814933776855
0 2
term: 0.31566837430000305
1 2
term: 1.097031593322754
______________________________
result: 3.7982814610004425


In [22]:
quad_term = 0
# TODO: It is still something wrong here !
for i in range(f):
    for j in range(i+1,f):
        vi_fj = tf.nn.embedding_lookup_sparse(W[:,i,:],sp_feat,sp_value,combiner="sum")
        vj_fi = tf.nn.embedding_lookup_sparse(W[:,j,:],sp_feat,sp_value,combiner="sum")
        term = tf.reduce_sum(vi_fj * vj_fi,axis=1)
        res = sess.run(term)
        quad_term += res[0]
print("result:quad_term)

-2.6580039858818054


In [12]:
res.shape

TensorShape([Dimension(None), Dimension(2)])